# IMPORTS

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.cluster import KMeans



## LEITURA DOS DADOS

In [19]:
df = pd.read_csv('../../RawData/train_radiomics_hipocamp.csv') 
df_test = pd.read_csv('../../RawData/test_radiomics_hipocamp.csv')
x_train = df.drop(columns=['Transition'])

# Tratamento

### Single Value Colums 

In [20]:
df = df.loc[:, df.nunique() > 1]


#  Analise das colunas que tem menos de 50 valores unicos 
n = df.nunique()
#for col, e in n.items():
#    if e < 50:  
#        print(f"Coluna: {col}, Valores Unicos : {e}")

### AGE BINING  ( secalhar isto era no 2 )

In [21]:

age_bins = [0, 65, 75, 85, 100]
# BINS_SIZER = ['<65', '65-74', '75-84', '85+']
age_labels = [60, 70, 80, 90] # VALOR MEDIO DO BIN 
df['Age'] = pd.cut(df['Age'], bins=age_bins, labels=age_labels).astype(int)

#print(df['Age'])

### Colunas Categoricas 

In [22]:
# Analisar a contagem de valores únicos para cada coluna categórica

# Identificar as colunas categóricas
categorical_columns = df.select_dtypes(include=['object', 'category']).columns

#for col in categorical_columns:
#    print(f"\nColuna: {col}")

#colunas_catagoricas_a_remover = ['ID', 'Image', 'Mask', 'diagnostics_Image-original_Hash', 'diagnostics_Mask-original_Hash'] 

colunas_catagoricas_a_remover = ['ID', 'Image', 'Mask', 'diagnostics_Image-original_Hash', 'diagnostics_Mask-original_Hash', 'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_CenterOfMassIndex', 'diagnostics_Mask-original_CenterOfMass'] 

# ** Bounding Box
#
# ** as colunas do 'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_CenterOfMassIndex', 'diagnostics_Mask-original_CenterOfMass'
# ** Deveriam ser retiradas, mas o bounding box pode ser importante para a zona de maior ativação do Alzimeir 
# **  ja a de centro de maxima devem ser muito correlacionados, por isso devem ser retirados mais para a frente 
# *TODO acabei por retirar para correr melhor os modelos, mas analisar se se deve retirar ou nao 

df.drop(columns=colunas_catagoricas_a_remover,axis= 1 , inplace= True)


### Eliminar Transition CN-MCI 

Como este tipo de precisao não vai ser realiza, decidimos retirar todas as linhas com ela relacinadas para nao confundir o nosso modelo

In [23]:
df = df[df['Transition'] != 'CN-MCI']

### Label Encoding

In [24]:
label_mapping = {
    'CN-CN': 0,
    'AD-AD': 1,
    'MCI-AD': 2,
    'MCI-MCI': 3,
#    'CN-MCI' : 4
}

df['Transition'] = df['Transition'].map(label_mapping)

## Correlacao com target 

In [25]:
def remove_highly_correlated_with_target(data, target_column, threshold):
    # Compute the correlation of each column with the target column
    corr_with_target = data.corr()[target_column].abs()
    
    # Identify columns to remove based on correlation with the target
    to_drop = corr_with_target[corr_with_target >= threshold].index
    to_drop = to_drop.drop(target_column)  # Remove target column itself if included
    
    # Drop the identified columns
    data_final = data.drop(columns=to_drop)

    return data_final
print(df.shape)
df = remove_highly_correlated_with_target(df, 'Transition',threshold=0.18)
print(df.shape)

(295, 2014)
(295, 1661)


## Correlacao colunas

In [26]:
def remove_highly_correlated_features(data, threshold):
    # Compute the correlation matrix
    corr_matrix = data.corr().abs()

    # Identify columns to remove based on correlation
    to_drop = set()
    for i in range(len(corr_matrix.columns)):
        for j in range(i + 1, len(corr_matrix.columns)):
            if corr_matrix.iloc[i, j] >= threshold:
                to_drop.add(corr_matrix.columns[j])

    # Drop the identified columns
    data_final = data.drop(columns=to_drop)

    return data_final



print(df.shape)
df = remove_highly_correlated_features(df,0.95)
print(df.shape)

(295, 1661)
(295, 843)


# Models

In [27]:

#experiment = setup(df, target='Transition')

In [28]:
#best = compare_models()

Com o tratamento atual, o pycaret diz que os melhores modelos são :
- Random Forest Classifier
- Extra Trees Classifier
- Gradient Boosting Classifier 
- K Neighbors Classifier
- Extreme Gradient Boosting 


### Aplicação dos modelos (default) ao dataset tratado

In [29]:


X = df.drop('Transition', axis=1)
y = df['Transition']


# Dividir o conjunto de dados em treino e teste com estratificação
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=34, stratify=y)


# Inicializar dicionário para armazenar as pontuações de cross-validation do modelo
model_cross_score = {}


### Funcao Matrix de Confusao 

In [30]:
def plot_confusion_matrix_with_labels(confusion_matrix, label_mapping):
    # Criar um mapeamento inverso
    reverse_label_mapping = {v: k for k, v in label_mapping.items()}
    
    # Aplicar o mapeamento inverso na matriz de confusão
    cm_with_labels = np.zeros_like(confusion_matrix, dtype=object)
    for i in range(confusion_matrix.shape[0]):
        for j in range(confusion_matrix.shape[1]):
            cm_with_labels[i, j] = f"{reverse_label_mapping[i]} (Pred: {reverse_label_mapping[j]})"

    # Criar um DataFrame para facilitar a visualização
    df_cm = pd.DataFrame(confusion_matrix, index=reverse_label_mapping.values(), columns=reverse_label_mapping.values())
    
    # Plotar a matriz de confusão
    plt.figure(figsize=(10, 7))
    sns.heatmap(df_cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix with Labels')
    plt.show()

#### Random Forest Classifier

In [31]:

rf_model = RandomForestClassifier(n_estimators=100, max_features='sqrt', random_state=34)

rf_score = cross_val_score(rf_model, X_train, y_train, cv=10,scoring='f1_macro')
print(rf_score)

model_cross_score['random_forest'] = rf_score





[0.37563131 0.42222222 0.36511438 0.420671   0.16666667 0.46806319
 0.40714286 0.49074675 0.39436381 0.46103896]


#### Extra Trees Classifier

In [32]:
et_model = ExtraTreesClassifier(criterion='gini', max_depth=20, random_state=34)

et_score = cross_val_score(et_model, X_train, y_train, cv=10,scoring='f1_macro')

print(et_score)

model_cross_score['Extra Trees'] = et_score

[0.27916667 0.42647059 0.49146825 0.45436508 0.23809524 0.6092033
 0.40714286 0.38720103 0.42847222 0.41382576]


#### Gradient Boosting Classifier 

In [33]:
gb_model = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100, random_state=34)

score_cross = cross_val_score(gb_model,X_train,y_train,cv=10,scoring='f1_macro')

print(score_cross)

model_cross_score['Gradient_boosting'] = score_cross



[0.2960275  0.3375     0.42777778 0.47786132 0.50575397 0.41066919
 0.23809524 0.38422688 0.56111111 0.34752747]


#### K Neighbors Classifier

In [34]:
kn_model = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto')
k_night_croos_score = cross_val_score(kn_model, X_train, y_train, cv=10,scoring='f1_macro')

print("Acuracy k night", k_night_croos_score)


model_cross_score['k_night'] = k_night_croos_score




Acuracy k night [0.31345029 0.46875    0.33554028 0.40296631 0.36507937 0.42205882
 0.1993007  0.26764706 0.23796791 0.35952381]


#### Extreme Gradient Boosting

In [35]:
xgb_model = XGBClassifier(n_estimators=100, max_depth=4, learning_rate=0.05,colsample_bytree=0.4, subsample=0.8, random_state=34)


score_cross = cross_val_score(xgb_model,X_train,y_train,cv=10,scoring='f1_macro')

print(score_cross)

model_cross_score['XGBoost'] = score_cross



[0.32361111 0.32222222 0.43125    0.43181818 0.3297619  0.35504202
 0.3419289  0.36098901 0.53321678 0.33831169]


SVC

In [36]:


svc_moldel = SVC(kernel='rbf', random_state=34)


score_cross = cross_val_score(svc_moldel,X_train,y_train,cv=10,scoring='f1_macro')

print(score_cross)

model_cross_score['SVC'] = score_cross



[0.21467391 0.18043478 0.29047619 0.22527473 0.17857143 0.23611111
 0.22142857 0.20075758 0.40217391 0.37371795]


In [37]:
def print_model_scores(model_scores):
    for model, scores in model_scores.items():
        max_score = np.max(scores)
        min_score = np.min(scores)
        diff = max_score - min_score
        avg_score = np.mean(scores)
        
        print(f"{model} (Max: {max_score:.4f}, Min: {min_score:.4f}, Diff: {diff:.4f}):")
        print(f"  Scores: {', '.join([f'{score:.4f}' for score in scores])}")
        print(f"  Average: {avg_score:.4f}")
        print('-' * 50)

# Chamar a função para imprimir o dicionário formatado
print_model_scores(model_cross_score)

random_forest (Max: 0.4907, Min: 0.1667, Diff: 0.3241):
  Scores: 0.3756, 0.4222, 0.3651, 0.4207, 0.1667, 0.4681, 0.4071, 0.4907, 0.3944, 0.4610
  Average: 0.3972
--------------------------------------------------
Extra Trees (Max: 0.6092, Min: 0.2381, Diff: 0.3711):
  Scores: 0.2792, 0.4265, 0.4915, 0.4544, 0.2381, 0.6092, 0.4071, 0.3872, 0.4285, 0.4138
  Average: 0.4135
--------------------------------------------------
Gradient_boosting (Max: 0.5611, Min: 0.2381, Diff: 0.3230):
  Scores: 0.2960, 0.3375, 0.4278, 0.4779, 0.5058, 0.4107, 0.2381, 0.3842, 0.5611, 0.3475
  Average: 0.3987
--------------------------------------------------
k_night (Max: 0.4688, Min: 0.1993, Diff: 0.2694):
  Scores: 0.3135, 0.4688, 0.3355, 0.4030, 0.3651, 0.4221, 0.1993, 0.2676, 0.2380, 0.3595
  Average: 0.3372
--------------------------------------------------
XGBoost (Max: 0.5332, Min: 0.3222, Diff: 0.2110):
  Scores: 0.3236, 0.3222, 0.4312, 0.4318, 0.3298, 0.3550, 0.3419, 0.3610, 0.5332, 0.3383
  Average